#### Challenge 41: Implement unpadded message recovery oracle

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [4]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import cryptopals as cp

<div class="alert alert-block alert-info">

Nate Lawson says we should stop calling it "RSA padding" and start calling it "RSA armoring". Here's why.

Imagine a web application, again with the Javascript encryption, taking RSA-encrypted messages which (again: Javascript) aren't padded before encryption at all.

You can submit an arbitrary RSA blob and the server will return plaintext. But you can't submit the same message twice: let's say the server keeps hashes of previous messages for some liveness interval, and that the message has an embedded timestamp:

```
{
  time: 1356304276,
  social: '555-55-5555',
}
```

You'd like to capture other people's messages and use the server to decrypt them. But when you try, the server takes the hash of the ciphertext and uses it to reject the request. Any bit you flip in the ciphertext irrevocably scrambles the decryption.

This turns out to be trivially breakable:

- Capture the ciphertext C
- Let N and E be the public modulus and exponent respectively
- Let S be a random number > 1 mod N. Doesn't matter what.
- Now:
    
    ```
    `C' = ((S**E mod N) C) mod N
    ```
    
- Submit C', which appears totally different from C, to the server, recovering P', which appears totally different from P
- Now:

              P'
        P = -----  mod N
              S

Oops!

Implement that attack.
    
<div class="alert alert-block alert-warning">    
    
#### **Careful about division in cyclic groups.**
    
Remember: you don't simply divide mod N; you multiply by the multiplicative inverse mod N. So you'll need a modinv() function.
    
</div>
</div>

In [5]:
key_size = 1024
[e,d,n] = cp.genRSA_keypair(key_size)
m = int((b'This is ridiculous! There\'s no such thing as privacy anymore!'*4).hex(), 16)
c = pow(m, e, n)

In [12]:
s = 79
c_ = (pow(s, e, n) * c) % n
p_ = pow(c_, d, n)

Despite the warning, I wanted to try straight division (not multiplying by the inverse mod N):

In [13]:
recovered_pt = (p_ // s) % n

print(bytes.fromhex(hex(recovered_pt)[2:]).decode())

This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!


Now the way it says to do it:

In [14]:
mi = cp.invmod(s, n)
recovered_pt = (p_*mi) % n
print(bytes.fromhex(hex(recovered_pt)[2:]).decode())

This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!This is ridiculous! There's no such thing as privacy anymore!


Let's try to break the division approach:

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [44]:
key_size = 1024
for ii in range(1000):
    
    print('.')
    [e,d,n] = cp.genRSA_keypair(key_size)
    c = pow(m, e, n)
    s = random.randint(0, 2**128-1)
    c_ = (pow(s, e, n) * c) % n
    p_ = pow(c_, d, n)
    recovered_pt = (p_ // s) % n
    
    if recovered_pt != m:        
        print('Broke it!')
        print(f'e={e}\nd={d}\nn={n}\ns={s}\n')
        break


.
Broke it!
e=3
d=13784155787454640965203320918456828855576350957154037222797011255165358470367881823584041715061754827774338504096176436120475191939229449781152894319605928252913300736282366016848779083912182646827542663181074178251488178035165720461277396960286993903600420480331825348205323534468578466963075371691051312181999979240211075743135335667166862700410731000271406232507052029364966540907676351362097390400954839239373713919572846050908107205998352793777885820373061841666941478495929055051905449803604551103700362868106112893945484374599667847712517441128352550514766396104086948998499666483037133089815898176502824916123
n=20676233681181961447804981377685243283364526435731055834195516882748037705551822735376062572592632241661507756144264654180712787908844174671729341479408892379369951104423549025273168625868273970241313994771611267377232267052748580691916095440430490855400630720497738022307985301702867700444613057536576968273287661965976930946948637139294611572506276884739532485

Here are two trivial examples of where division by a # and multiplication by a multiplicative inverse are 1) the same, and 2) different within a cyclic group (modulo arithmetic).

In [43]:
n = 17
s = 4
x = 12

print((x//s) % n)
print((x * cp.invmod(s, n)) % n)

n = 17
s = 4
x = 9

print((x//s) % n)
print((x * cp.invmod(s, n)) % n)

3
3
2
15
